In [1]:
import dicom as dcm
import SimpleITK as sitk
import numpy as np 
import array
from numpy import unravel_index
import matplotlib.pyplot as plt
%matplotlib inline

from __future__ import print_function
from ipywidgets import interact

In [2]:
def ct_viewer(im):
    def plot_image(myslice):
        plt.imshow(im[myslice])#, cmap=plt.cm.gray
    interact(plot_image, myslice=(0, im.shape[0]))
    return;

In [3]:
#read rt dose dicom file
#rd=dcm.read_file('RD.201700806.Dose_LIT Tumoral_LIT Tumoral.dcm')
rd=dcm.read_file('new3/rd_joue.dcm')
study=str(rd.PatientName.split('^')[0])
#print(rd)
#get pixel array
rdpix=rd.pixel_array.astype(np.float32)
print(unravel_index(rdpix.argmax(), rdpix.shape))
#create an sitk image from this array
rdim=sitk.GetImageFromArray(rdpix)

#Set origin of image from rt dose value
origin=[]
for val in rd.ImagePositionPatient: origin.append(float(val))
rdim.SetOrigin(origin)

#Set spacing of image from rt dose value (x et y)
spacing=[]
for val in rd.PixelSpacing: spacing.append(float(val))

#for z direction
spacez=float(rd.GridFrameOffsetVector[1]-rd.GridFrameOffsetVector[0])
spacing.append(spacez)
rdim.SetSpacing(spacing)

#write rd image in '.raw'
#sitk.WriteImage(rdim, 'rd_boobs.mhd')
sitk.WriteImage(rdim, 'new3/eclipse_'+study+'.mhd')

(75, 18, 76)


In [4]:
#Get the rd raw created  and gate image
#refim=sitk.ReadImage('rd_boobs.mhd')
#gateim=sitk.ReadImage('boobs1.mhd')

refim=sitk.ReadImage('new3/eclipse_'+study+'.mhd')
gateim=sitk.ReadImage('new3/JoueG_test1.2.mhd')


# get pixels arrays of images and vizualization
gatepix=sitk.GetArrayFromImage(gateim)
refpix=sitk.GetArrayFromImage(refim)
print(unravel_index(gatepix.argmax(), gatepix.shape))
print(unravel_index(refpix.argmax(), refpix.shape))
#Show Info and vizualization
print('rd origin: ', refim.GetOrigin())
print('rd spacing: ',refim.GetSpacing())

print('rd size: ',   refim.GetSize())
ct_viewer(refpix)
print('gate origin: ',gateim.GetOrigin())
print('gate sapcing: ',gateim.GetSpacing())
print('gate size: ',gateim.GetSize())
ct_viewer(gatepix)

(78, 25, 70)
(75, 18, 76)
rd origin:  (-226.738281, -136.738281, -175.0)
rd spacing:  (2.5, 2.5, 2.5)
rd size:  (181, 130, 108)


interactive(children=(IntSlider(value=54, description=u'myslice', max=108), Output()), _dom_classes=('widget-interact',))

gate origin:  (-200.332, -154.238, -175.0)
gate sapcing:  (2.5, 2.5, 2.5)
gate size:  (164, 128, 124)


interactive(children=(IntSlider(value=62, description=u'myslice', max=124), Output()), _dom_classes=('widget-interact',))

In [39]:
#Resample (Downsample) rd image into gate image size an resolution
refim_res=sitk.Resample(refim, gateim, sitk.Transform(), sitk.sitkNearestNeighbor, 0)
#g2=sitk.Resample(gateim, refim, sitk.Transform(), sitk.sitkNearestNeighbor, 0)
#Get pixel array

#Conversion en Gy

refim_respix=sitk.GetArrayFromImage(refim_res)
refim_respix=refim_respix*4.0*10**-6
print(refim_respix.max())
refim_res2=sitk.GetImageFromArray(refim_respix)
refim_res2.SetOrigin(refim_res.GetOrigin())
refim_res2.SetSpacing(refim_res.GetSpacing())

#g2pix=sitk.GetArrayFromImage(g2pix)
#print(refim_respix.shape)
#print(gatepix.shape)
ct_viewer(refim_respix)

4.52587


interactive(children=(IntSlider(value=62, description=u'myslice', max=124), Output()), _dom_classes=('widget-interact',))

In [26]:
#Sow info and vizualization
print('rd resampled origin: ',refim_res.GetOrigin())
print('rd resampled spacing: ',refim_res.GetSpacing())
print('rd resampled size: ',refim_res.GetSize())
#bons=[-201.582, 152.988, -113.75]
#refim_res.SetOrigin(bons)
ct_viewer(refim_respix)
#refim_respix=sitk.GetArrayFromImage(refim_res)

rd resampled origin:  (-200.332, -154.238, -175.0)
rd resampled spacing:  (2.5, 2.5, 2.5)
rd resampled size:  (164, 128, 124)


interactive(children=(IntSlider(value=62, description=u'myslice', max=124), Output()), _dom_classes=('widget-interact',))

In [40]:
#Write resampled image
#sitk.WriteImage(refim_res, 'rd_boobs_resampled.mhd')
sitk.WriteImage(refim_res2, 'new3/eclipse_'+study+'_resampled.mhd')

In [20]:
#Create a boolean mask to apply on Gate image (remove dose outside the patient according to RTdose)
mask=np.ma.make_mask(refim_respix)
gatepix=gatepix[:, :, :]
#apply on gate pixel array
gatepix_mask=gatepix*mask
gatepix_mask=gatepix_mask[:,:,:]

ct_viewer(mask)
ct_viewer(gatepix_mask)

interactive(children=(IntSlider(value=62, description=u'myslice', max=124), Output()), _dom_classes=('widget-interact',))

interactive(children=(IntSlider(value=62, description=u'myslice', max=124), Output()), _dom_classes=('widget-interact',))

In [21]:
#Create a raw image of gate with mask

#conversion en Dose Abs. 
gatepix_mask=gatepix_mask*(1.0/0.000140945)*(547.0/100.0)

gate_dose=sitk.GetImageFromArray(gatepix_mask)
gate_dose.SetOrigin(gateim.GetOrigin())
gate_dose.SetSpacing(gateim.GetSpacing())
print(gate_dose.GetOrigin())
print(gate_dose.GetSpacing())
print(gate_dose.GetSize())
#sitk.WriteImage(gate_dose, 'gate_dose.mhd')
sitk.WriteImage(gate_dose, 'new3/gate_'+study+'.mhd')

(-200.332, -154.238, -175.0)
(2.5, 2.5, 2.5)
(164, 128, 124)


In [ ]:
#Create rtdose.dcm fil with gate and rt dose resampled image

#need a empty matrix (of size of original image) to fill up with our images
#z, y, x why ???
Matrix=np.zeros(((rdim.GetSize()[2], rdim.GetSize()[1], rdim.GetSize()[0])))

lowx=(Matrix.shape[0] // 2) - (gatepix_mask.shape[0] //2) -1
upx= (Matrix.shape[0] // 2) + (gatepix_mask.shape[0] //2)
lowy=(Matrix.shape[1] // 2) - (gatepix_mask.shape[1] //2)
upy= (Matrix.shape[1] // 2) + (gatepix_mask.shape[1] //2)
lowz=(Matrix.shape[2] // 2) - (gatepix_mask.shape[2] //2)
upz= (Matrix.shape[2] // 2) + (gatepix_mask.shape[2] //2)

#use slicing to replace the center of the matrix by our image
Matrix[lowx:upx, lowy:upy, lowz:upz]=gatepix_mask

#Pixel was written in int32 we have float32 so we multiply to have inerger with enough precison to don't lose physcal information (the troncature of float lose less information than the uncertainty of gate simulation)
scale=1e-10
Matrix=Matrix/scale
Matrix=Matrix.astype(np.int32)
#ct_viewer(Matrix)

#insert dose factor scaling into rt dose
rd.DoseGridScaling=scale

#write matrix into pixel data and save a new dicom file
rd.PixelData = Matrix.tostring()
rd.save_as("gate_rtdose.dcm")

In [ ]:
#Same operation for rt dose resmpled image
#Create rtdose.dcm fil with gate and rt dose resampled image

#need a empty matrix (of size of original image) to fill up with our images
#z, y, x why ???
Matrix=np.zeros(((rdim.GetSize()[2], rdim.GetSize()[1], rdim.GetSize()[0])))

lowx=(Matrix.shape[0] // 2) - (refim_respix.shape[0] //2) -1
upx= (Matrix.shape[0] // 2) + (refim_respix.shape[0] //2)
lowy=(Matrix.shape[1] // 2) - (refim_respix.shape[1] //2)
upy= (Matrix.shape[1] // 2) + (refim_respix.shape[1] //2)
lowz=(Matrix.shape[2] // 2) - (refim_respix.shape[2] //2)
upz= (Matrix.shape[2] // 2) + (refim_respix.shape[2] //2)

#use slicing to replace the center of the matrix by our image
Matrix[lowx:upx, lowy:upy, lowz:upz]=refim_respix

#Pixel was written in int32 we have float32 so we multiply to have inerger with enough precison to don't lose physcal information (the troncature of float lose less information than the uncertainty of gate simulation)
#scale=1e-10
#Matrix=Matrix/scale
Matrix=Matrix.astype(np.int32)
#ct_viewer(Matrix)

#insert dose factor scaling into rt dose
#rd.DoseGridScaling=scale

#write matrix into pixel data and save a new dicom file
rd.PixelData = Matrix.tostring()
rd.save_as("rtdose_linac6e10x10_resampled.dcm")

In [ ]:
normemcim=sitk.ReadImage('rd_boobs.mhd')
normgateim=sitk.ReadImage('gate_dose.mhd')
normemcimpix=sitk.GetArrayFromImage(normemcim)
normgateimpix=sitk.GetArrayFromImage(normgateim)
normemcimpix=normemcimpix/(normemcimpix.max())
normgateimpix=normgateimpix/(normgateimpix.max())
out1=sitk.GetImageFromArray(normemcimpix)
out2=sitk.GetImageFromArray(normgateimpix)
sitk.WriteImage(out1, 'emcnorm.mhd')
sitk.WriteImage(out2, 'gatenorm.mhd')
print(unravel_index(normemcimpix.argmax(), normemcimpix.shape))
print(unravel_index(normgateimpix.argmax(), normgateimpix.shape))

In [ ]:
mask1=np.ma.make_mask(refim_respix)
imtest=sitk.ReadImage('gate_ct.mhd')
imtestpix=sitk.GetArrayFromImage(imtest)
mask2=np.ma.make_mask(imtestpix)
M=np.zeros(((mask1.shape[0], mask1.shape[1], mask1.shape[2])))
M1=(M+1.0)*mask1
M2=(M+1.0)*mask2
M1.astype(np.float32)
M2.astype(np.float32)
mask1im=sitk.GetImageFromArray(M1)
mask2im=sitk.GetImageFromArray(M2)
sitk.WriteImage(mask1im, 'm1.mhd')
sitk.WriteImage(mask2im, 'm2.mhd')